<a href="https://colab.research.google.com/github/Ramanarayanan/LLM-Finetuning/blob/main/Unsloth_Qwen2_5_1_5B_bnb_4bit_fineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth" "trl" "datasets" "accelerate" --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
# from datasets import load_dataset

# # Replace with path/URL to your own data
# dataset = load_dataset("json", data_files={"train": "train.jsonl"}, split="train")
# print(dataset[0])

from datasets import Dataset

# Replace or expand this list with your own data
data = {
    "instruction": [
        "What is the capital of Chennai City?",
        "Who wrote Romeo and Juliet?",
        "Sum 17 and 25.",
    ],
    "output": [
        "Ambattur.",
        "William Shakespeare.",
        "The sum is 42.",
    ]
}

dataset = Dataset.from_dict(data)
print(dataset)
print(dataset[0])



Dataset({
    features: ['instruction', 'output'],
    num_rows: 3
})
{'instruction': 'What is the capital of Chennai City?', 'output': 'Ambattur.'}


In [23]:
def formatting_func(examples):
    # Each examples is a dict of columns: {'instruction':..., 'output':...}
    # Return a list of joined input-output strings (one for each sample)
    return [
        f"Instruction: {instr}\nResponse: {resp}"
        for instr, resp in zip(examples["instruction"], examples["output"])
    ]


In [2]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23

In [24]:
from unsloth import FastLanguageModel

MODEL_NAME = "unsloth/Qwen2.5-1.5B-bnb-4bit"  # Or: "unsloth/Qwen1.5-1.5B-Chat-bnb-4bit"
MAX_SEQ_LENGTH = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)



==((====))==  Unsloth 2025.8.1: Fast Qwen2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [25]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                       # LoRA rank, tune as needed
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],  # Standard for Qwen
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = MAX_SEQ_LENGTH,
    use_rslora = False,
)


In [36]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    formatting_func=formatting_func,
    args = SFTConfig(
        max_length = MAX_SEQ_LENGTH,
        per_device_train_batch_size = 2,         # Start small for Colab
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,                          # Increase for real training
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
        num_train_epochs = 4

    ),
)
trainer.train()


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)


Step,Training Loss
1,1.272700
2,1.272700
3,1.239700
4,1.171500
5,1.062400
6,0.920700
7,0.762300
8,0.759800
9,0.638400
10,0.551600


TrainOutput(global_step=60, training_loss=0.24386379445592563, metrics={'train_runtime': 55.2702, 'train_samples_per_second': 8.685, 'train_steps_per_second': 1.086, 'total_flos': 25560900212736.0, 'train_loss': 0.24386379445592563})

In [39]:
# Hugging Face save format
model.save_pretrained("outputs/finetuned_qwen")
tokenizer.save_pretrained("outputs/finetuned_qwen")


('outputs/finetuned_qwen/tokenizer_config.json',
 'outputs/finetuned_qwen/special_tokens_map.json',
 'outputs/finetuned_qwen/vocab.json',
 'outputs/finetuned_qwen/merges.txt',
 'outputs/finetuned_qwen/added_tokens.json',
 'outputs/finetuned_qwen/tokenizer.json')

In [27]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [42]:
model.push_to_hub("RamAi24/Qwen2.5-1.5B-bnb-4bi_gpu_test")
tokenizer.push_to_hub("RamAi24/Qwen2.5-1.5B-bnb-4bi_gpu_test")

Saved model to https://huggingface.co/RamAi24/Qwen2.5-1.5B-bnb-4bi_gpu_test


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/RamAi24/Qwen2.5-1.5B-bnb-4bi_gpu_test/commit/fad6b785f0d48bcc300bc11e4fd1dc9f305fb82b', commit_message='Upload tokenizer', commit_description='', oid='fad6b785f0d48bcc300bc11e4fd1dc9f305fb82b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RamAi24/Qwen2.5-1.5B-bnb-4bi_gpu_test', endpoint='https://huggingface.co', repo_type='model', repo_id='RamAi24/Qwen2.5-1.5B-bnb-4bi_gpu_test'), pr_revision=None, pr_num=None)

In [38]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from unsloth import FastLanguageModel

model_dir = "outputs/finetuned_qwen"  # update if your path is different

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the model using FastLanguageModel for compatibility with unsloth optimizations
model, _ = FastLanguageModel.from_pretrained(
    model_name = model_dir,
    max_seq_length = MAX_SEQ_LENGTH, # Reuse the previously defined MAX_SEQ_LENGTH
    load_in_4bit = True, # Load in 4-bit as it was trained
)

==((====))==  Unsloth 2025.8.1: Fast Qwen2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [40]:
prompt = "Instruction: What is the capital of Chennai City?\nResponse:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")


In [41]:
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Model Output:")
    print(decoded)


Model Output:
Instruction: What is the capital of Chennai City?
Response: Ambattur.
